In [143]:
import numpy 
import pandas
import nltk
import csv
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

##### Los topicos se encuentran juntos en el dataframe, esta función los separa de acuerdo al tema

In [144]:
def separaTopico(topico, df): #Separa todos los encabezados de acuerdo a su tópico
    listTopic = []
    dataArr = numpy.asarray(df)
    for i in range(len(dataArr)):
        if(dataArr[i][1] == topico):
            listTopic.append(dataArr[i][0])
    return listTopic

##### Cuando se tienen las palabras unicas de cada tópico, se eliminan caracteres especiales.

In [145]:
def limpiaTexto(text):
    "List all the word tokens in a text."
    return re.findall('[a-zA-Z0-9\-áéíóúÁÉÍÓÚ]{4,254}', text.lower())


##### Esta función busca eliminar palabras plurales para quedarnos únicamente con las singulares a través de la raíz de la palabra 

In [160]:
def RaizSingular(palabra):
    vocales = ['a', 'e', 'i', 'o', 'u']
    ultima = len(palabra) -1
    raiz = []
    plural = False

    if (palabra[ultima] == 's'): #Verificamos si la última letra es 's'
        if (palabra[ultima - 1] == 'a' or 
            palabra[ultima - 1] == 'i' or
            palabra[ultima - 1] == 'o' or
            palabra[ultima - 1] == 'u'):
            
            for i in range(ultima):
                    raiz.append(palabra[i])
            
            palabra = "".join(raiz)
            return palabra
            
        if(palabra[ultima - 1] == 'e'):
            if (palabra[ultima - 2] == 'c'):
                for i in range(ultima - 2):
                    raiz.append(palabra[i])
                    
                raiz.append('z')
                    
                palabra = "".join(raiz)
                return palabra
            
            elif (palabra[ultima - 2] != 'a' or
                palabra[ultima - 2] != 'e' or
                palabra[ultima - 2] != 'i' or
                palabra[ultima - 2] != 'o' or
                palabra[ultima - 2] != 'u'):
                
                for i in range(ultima - 1):
                    raiz.append(palabra[i])
                    
                palabra = "".join(raiz)
                return palabra
                                                                        
    else:
        return palabra

##### Cada encabezado corresponde a un documento, esta función separa cada palabra para agregarlas a un solo array

In [147]:
def PalabrasTopico(listTopico): #Devuelve todas las palabras que aparecen en los titulares de las noticias 
    listHeadLines = []
    for i in range(len(listTopico)):
        #aux = nltk.word_tokenize(arrayTopico[i])
        aux = limpiaTexto(listTopico[i])
        for j in range(len(aux)):
            aux2 = aux[j]
            listHeadLines.append(aux2)
            
    return numpy.asarray(listHeadLines)

In [148]:
def Probabilidad(palabrasTopico, palabrasUnicasTopico): #Devuelve un array con la ocurrencias de cada única del tópico
    Prob = numpy.zeros(len(palabrasUnicasTopico))
    for i in range(len(palabrasUnicasTopico)):
        Prob[i] = numpy.sum(palabrasTopico == palabrasUnicasTopico[i])
        Prob[i] = Prob[i] /len(palabrasTopico)
        
    return Prob

##### Recibe el vector de probabilidades y elige la máxima para finalmente devolver el tópico al que pertenece

In [149]:
def Clasificador(arrayProbabilidades):
    indice = numpy.where(arrayProbabilidades == max(arrayProbabilidades))[0][0]
    print("Indice = ", type(indice))
    print("Max = ", max(arrayProbabilidades))
    if (indice == 0):
        print("Covid")
    if (indice == 1):
        print("Tecnología")
    if (indice == 2):
        print("Deportes")
    if (indice == 3):
        print("Economía")
    if (indice == 4):
        print("Cultura")

In [150]:
def MAP(tweet, palabrasCovid, AlfabetoCovid, palabrasTecn, AlfabetoTecn, palabrasDep, AlfabetoDep):
    T = nltk.word_tokenize(tweet)
    Proba = numpy.ones(5)

    #Covid
    for j in range(len(T)):
        aux = numpy.sum(palabrasCovid == T[j]) + 1
        Proba[0] *= (aux + 1) / (len(AlfabetoCovid) + aux)
    
    #Tecnologia
    for j in range(len(T)):
        aux = numpy.sum(palabrasTecn == T[j]) 
        Proba[1] *= (aux + 1) / (len(AlfabetoTecn) + aux)

    #Deportes
    for j in range(len(T)):
        aux = numpy.sum(palabrasDep == T[j]) + 1
        Proba[2] *= (aux + 1) / (len(AlfabetoDep) + aux)
        
    #Economía
    for j in range(len(T)):
        aux = numpy.sum(palabrasEcon == T[j]) + 1
        Proba[3] *= (aux + 1) / (len(AlfabetoEcon) + aux)
        
    #Economía
    for j in range(len(T)):
        aux = numpy.sum(palabrasCult == T[j]) + 1
        Proba[4] *= (aux + 1) / (len(AlfabetoCult) + aux)

    for p in range(len(Proba)):
        Proba[p] = Proba[p] * 1/5
        
    return Clasificador(Proba)
        

##### Abrimos el dataset con pandas

In [151]:
nombres = ['Titular', 'Tópico']
dataTrain = pandas.read_csv('titulares.csv', names=nombres)
dataTrain = dataTrain.dropna()
dataTrain

,Titular,Tópico
0,"Pemex suma 251 decesos, mil 538 casos confirma...",covid
1,Semáforos para nueva normalidad tendrán “pulso...,covid
2,España inicia 10 días de luto por las víctimas...,covid
3,"“Pero qué necesidad”, dice Calderón sobre la f...",covid
4,OIT: el desempleo será mayor en países como Mé...,covid
...,...,...
4135,"“Balam Antsetik. La Segunda Era”, de Margarita...",cultura
4136,“The Angel”: Una promesa incumplida,cultura
4137,Iris Bringas con “Rockdrigo” en la Biblioteca ...,cultura
4138,"Gabinete Rococó, en el Museo Nacional de San C...",cultura


In [152]:
encabezadosCov = separaTopico('covid', dataTrain)
alfaCov = PalabrasTopico(encabezadosCov)
alfaCov

array(['pemex', 'suma', 'decesos', ..., 'mundo', 'murió', 'covid-19'],
      dtype='<U25')

##### Separamos del dataset por tópicos y a cada encabezado lo separamos por palabra de los documentos eliminando caracteres especiales

In [159]:
palabrasCovid = PalabrasTopico(separaTopico('covid', dataTrain))
AlfabetoCovid = numpy.unique(palabrasCovid)

palabrasTecn = PalabrasTopico(separaTopico('tecnologia', dataTrain))
AlfabetoTecn = numpy.unique(palabrasTecn)
#print(len(palabrasTecn))
#print(AlfabetoTecn)

palabrasDep = PalabrasTopico(separaTopico('deportes', dataTrain))
AlfabetoDep = numpy.unique(palabrasDep)

palabrasEcon = PalabrasTopico(separaTopico('economia', dataTrain))
AlfabetoEcon = numpy.unique(palabrasEcon)
print(len(palabrasEcon))
print(AlfabetoEcon)

palabrasCult = PalabrasTopico(separaTopico('cultura', dataTrain))
AlfabetoCult = numpy.unique(palabrasCult)

Alfabeto = numpy.concatenate((AlfabetoCovid, AlfabetoTecn, AlfabetoDep, AlfabetoEcon, AlfabetoCult))
#Alfabeto.shape

395
['2003' '2009' '2015' '2020' 'abril' 'acatará' 'accionistas' 'acuerdo'
 'acumulada' 'adquiere' 'advierte' 'afecta' 'afectará' 'afore' 'ahorra'
 'alerta' 'america' 'amlo' 'amodio' 'amparos' 'ante' 'anticorrupción'
 'anual' 'anuncia' 'aplaude' 'apoyos' 'atenta' 'aumentaron' 'aumentarán'
 'aumentó' 'baja' 'bajo' 'balanza' 'bank' 'banxico' 'barril' 'bbva' 'bofa'
 'caer' 'caigan' 'cambiario' 'campeche' 'casi' 'causa' 'cayó' 'caída'
 'cenace' 'cierra' 'circulación' 'cofece' 'coloca' 'comercial' 'comercio'
 'comercios' 'como' 'concluyen' 'condenan' 'consar' 'consumidores'
 'contra' 'contracción' 'convierten' 'corrupción' 'covid-19' 'creará'
 'crecimiento' 'crisis' 'criterio' 'cuatro' 'decreto' 'dejan' 'demanda'
 'depreciación' 'derechos' 'desde' 'desempleo' 'desplomará' 'desplome'
 'deuda' 'deudas' 'devaluación' 'devolución' 'diarios' 'diciembre'
 'digitales' 'disminuyó' 'déficit' 'dólar' 'dólares' 'economía' 'efectos'
 'elevada' 'eléctrico' 'empleos' 'empresarios' 'empresas' 'energético'

##### Obtenemos las probabilidades a priori, haciendo cada una de estas equiprobabloe $ P(C) $

In [154]:
PCovid = 1/3
PTecn = 1/3
PDep = 1/3
print("PCovid = ", PCovid, "PTecn = ", PTecn, "PDep = ", PDep)

PCovid =  0.3333333333333333 PTecn =  0.3333333333333333 PDep =  0.3333333333333333


##### Obtenemos la frecuencia de palabras de cada atributo (palabra) $ P(A_{i} | C_{j}) $

In [155]:
ProbaCov = Probabilidad(palabrasCovid, AlfabetoCovid)
print(ProbaCov)
ProbaTecn = Probabilidad(palabrasTecn, AlfabetoTecn)
ProbaDep = Probabilidad(palabrasDep, AlfabetoDep)
ProbaEcon = Probabilidad(palabrasEcon, AlfabetoEcon)
ProbaCult = Probabilidad(palabrasCult, AlfabetoCult)

[0.00025478 0.00012739 0.00012739 ... 0.00012739 0.00025478 0.00012739]


##### Pruebas

In [156]:
tweet = "petroleo"

In [157]:
MAP(tweet, palabrasCovid, AlfabetoCovid, palabrasTecn, AlfabetoTecn, palabrasDep, AlfabetoDep)

Indice =  <class 'numpy.int64'>
Max =  0.0014440433212996391
Economía
